# Agentic Essay Writer

In [1]:
from dotenv import load_dotenv

_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage

memory = MemorySaver()

In [3]:
class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int

In [4]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-5-nano", temperature=0)

In [5]:
PLAN_PROMPT = """You are an expert writer tasked with writing a high level outline of an essay. \
Write such an outline for the user provided topic. Give an outline of the essay along with any relevant notes \
or instructions for the sections."""

WRITER_PROMPT = """You are an essay assistant tasked with writing excellent 5-paragraph essays.\
Generate the best essay possible for the user's request and the initial outline. \
If the user provides critique, respond with a revised version of your previous attempts. \
Utilize all the information below as needed: 

------

{content}"""

REFLECTION_PROMPT = """You are a teacher grading an essay submission. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for length, depth, style, etc."""

RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can \
be used when writing the following essay. Generate a list of search queries that will gather \
any relevant information. Only generate 3 queries max."""

RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can \
be used when making any requested revisions (as outlined below). \
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max."""


In [18]:
from langchain_core.pydantic_v1 import BaseModel

class Queries(BaseModel):
    queries: List[str]

In [7]:
from tavily import TavilyClient
import os
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [8]:
def plan_node(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ]

    response = model.invoke(messages)
    return {"plan": response.content}

In [19]:
def research_plan_node(state: AgentState):
    messages = [
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ]

    queries = model.with_structured_output(Queries).invoke(messages)

    content = state['content'] or []

    for q in queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])

    return {"content": content}

In [23]:
def generation_node(state: AgentState):
    content = "\n\n".join(state['content'] or [])
    user_message = HumanMessage(
        content=f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}")
    messages = [
        SystemMessage(
            content=WRITER_PROMPT.format(content=content)
        ),
        user_message
        ]
    response = model.invoke(messages)
    return {
        "draft": response.content, 
        "revision_number": state.get("revision_number", 1) + 1
    }


In [24]:
def research_critique_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
        HumanMessage(content=state['critique'])
    ])
    content = state["content"] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [25]:
def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT), 
        HumanMessage(content=state["draft"])
    ]
    response = model.invoke(messages)
    return {"critique": response.content}

In [12]:
def should_continue(state):
    if state["revision_number"] > state["max_revisions"]:
        return END
    return "reflect"

In [26]:
builder = StateGraph(AgentState)

builder.add_node("planner", plan_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_critique", research_critique_node)

builder.set_entry_point("planner")

builder.add_conditional_edges("planner", should_continue, {END: END, "reflect": "reflect"})

builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")

builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")

graph=builder.compile(checkpointer=memory)

In [28]:
thread = {"configurable": {"thread_id": "1"}}
initial_state = {
    'task': "What is the best way to plan to build a house from scratch",
    'plan': "",
    'draft': "",
    'critique': "",
    'content': [],
    'max_revisions': 2,
    'revision_number': 1,
}

for s in graph.stream(initial_state, thread):
    print(s)

{'planner': {'plan': 'Title: From Vision to Reality: A Strategic Plan for Building a House from Scratch\n\nThesis (optional for essay): Building a house from scratch succeeds when you integrate clear goals, rigorous budgeting, site-aware design, regulatory compliance, skilled partnerships, and disciplined project management from the earliest stages through completion.\n\nOutline and notes for each section\n\n1) Introduction\n- Purpose: Set up the question of what “best way” means in the context of building a house from scratch.\n- Elements to cover:\n  - The aspirational side (dream home) and the practical side (costs, timelines, risks).\n  - Why a structured plan matters more than a great design alone.\n  - Your thesis statement or central argument about a holistic, phased planning approach.\n- Notes: Briefly preview the major sections of the essay (goals, finances, site/design, permits, procurement, construction management, risk, sustainability, and lessons).\n\n2) Clarify goals, sco

/Users/robert/.pyenv/versions/jupyter/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:1893: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


HTTPError: 422 Client Error: Unprocessable Entity for url: https://api.tavily.com/search

In [29]:
for s in graph.get_state_history(thread):
    print(s, '\n')

StateSnapshot(values={'task': 'What is the best way to plan to build a house from scratch', 'plan': 'Title: From Vision to Reality: A Strategic Plan for Building a House from Scratch\n\nThesis (optional for essay): Building a house from scratch succeeds when you integrate clear goals, rigorous budgeting, site-aware design, regulatory compliance, skilled partnerships, and disciplined project management from the earliest stages through completion.\n\nOutline and notes for each section\n\n1) Introduction\n- Purpose: Set up the question of what “best way” means in the context of building a house from scratch.\n- Elements to cover:\n  - The aspirational side (dream home) and the practical side (costs, timelines, risks).\n  - Why a structured plan matters more than a great design alone.\n  - Your thesis statement or central argument about a holistic, phased planning approach.\n- Notes: Briefly preview the major sections of the essay (goals, finances, site/design, permits, procurement, constr